# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,datetime
import re

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
# ---------- 关键设置：让长文本不截断 ----------
# 设置列的最大宽度为“无限制”（None 或 -1，确保长字符串完整显示）
pd.set_option('display.max_colwidth', None)  

# ---------- 可选设置（按需开启） ----------
# 设置最大显示行数（None 表示显示所有行，若数据行数极多，可设为具体数值）
pd.set_option('display.max_rows', 20)  

# 允许 DataFrame 横向展开显示（避免列折叠）
pd.set_option('display.expand_frame_repr', True) 
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
def select_data(Time):
    sql1 = f''' -- 订单&风控信息  近10日数据   
    SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
    ,om.status, om.user_id
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
    when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
    when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
    ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
    ,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
    ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
    ,om.user_mobile,tmu.true_name,tmu.id_card_num
    ,top.total_describes,tor.decision_result,om.cancel_reason
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
    ,cc.name as channel_name         -- 来源渠道
    ,cc.channel_type_id              -- 渠道id
    ,pa.name as activity_name        -- 活动名称
    ,om.merchant_id,om.merchant_name
    ,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
    ,om.buy_service_product,tso.status as service_status 
    ,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
    , tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
    , om.order_type, om.union_rent_tag, tojo.app_type, tprm.rent_received 已收租金 
    , tolog.status 物流状态
    ,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
    ,tp.classify_id, tpmn.name 机型
    from  db_digua_business.t_order  om
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id
    -- 备注信息合并 
    left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
    on om.id = top.order_id 
    -- 服务信息
    left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
    -- 渠道名称
    left join db_digua_business.t_channel cc on om.channel = cc.scene 
    -- 活动名称
    left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
    -- 用户信息 
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    -- 商品信息
    left join db_digua_business.t_order_details tod on om.id = tod.order_id
    -- 免押信息  
    left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
    from db_digua_business.t_order_pay t 

    where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
    on topay.order_id=om.id   and  topay.rn = 1 
    -- 商家订单转移表
    left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
    -- 京东外部订单关联表
    left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
    -- 订单物流表
    left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
    -- 租后分期表
    -- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    left join db_credit.risk risk on risk.order_id=om.id
    -- 商品表
    left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB

    left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id


    where om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
    -- and pa.type!=4
    and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -9 day )               -- 近一周数据
    -- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-11-17'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-11-23'
    -- and hour(om.create_time) >= 11
    and TIME(om.create_time) <= '{Time}'

    ;
    '''
    # .format(time)

    df_order = clean.query(sql1)
    df_order.shape
    # sql3 = '''
    # SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
    # '''
    # df_risk_examine = clean.query(sql3)
    
    sql_risk = ''' -- risk等级
                        select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
                        from db_credit.risk
                        '''
    df_risk = clean.query(sql_risk)
    sql3 = '''
                SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
                '''
    df_risk_examine = clean.query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

    sql_ra = ''' -- 996强拒表
                select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
                '''
    df_ra = clean.query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
    return df_order, df_risk_examine, df_risk, df_ra, df_re

In [5]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)

# 数据处理

In [6]:
def process_data(df_order, df_risk, df_ra, df_re):
    # 复制数据
    df = df_order.copy()
    # df = df[df.type!=4]
    
    # 处理日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    print(df["下单日期"].unique())
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

    # 处理备注信息
    # df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
    def getcolor(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="颜色":
                return color_list[j]["value"]
    df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)


    def getneicun(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="内存":
                return color_list[j]["value"]

    df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
    def getfangan(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="租赁方案":
                return color_list[j]["value"]
    df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)

    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")


    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
    df.loc[:,'机型内存'] = df['机型'] +'_'+ df['内存']
    
    df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
    
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
    
    # 订单去重
    df = clean.order_drop_duplicates(df)
    
    # 定义状态
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    # df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
    # df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
    df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
    
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
    
    # 保留、剔除商家数据
    df_contain = df.copy()
    df = clean.drop_merchant(df)
    # 去重订单明细导出
    # today = datetime.today() 
    # t_date = today.date().strftime('%Y-%m-%d')
    
    df_j = df[df["进件"]=="进件"]
    
    df = clean.status_node(df)
    df.loc[:, "拒量订单数"] = np.where((df['机审通过件']==1)&(df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True), 1 , 0)
    
    # 剔除拒量订单
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    day_name_cn_mapping = {
        'Monday': '星期一',
        'Tuesday': '星期二',
        'Wednesday': '星期三',
        'Thursday': '星期四',
        'Friday': '星期五',
        'Saturday': '星期六',
        'Sunday': '星期日'
    }
    
    return df, df2

# 分组聚合

In [7]:
def group_data(df0, classify):
    df_group = df0.groupby(classify).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum', '拒量订单数':'sum'})
    df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["进件数"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group['进件数']
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["进件数"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["进件数"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group['进件数']
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group['进件数']
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group['进件数']
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group['进件数']
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group['进件数']
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["出库"]/df_group['进件数']
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

In [8]:
# def data_group0(self, df, df2, df_risk_examine, model):
#         '''
#         计算df按model分组的数据
#         :param df: 包含小蚂蚁租机的df
#         :param df2: 不含小蚂蚁租机的df
#         :param df_risk_examine: 包含出库前风控强拒的df
#         :param model: 需要分组的类型
#         :return: 返回分组后的数据
#         '''
#         # 包含拒量和联合拒量订单|支付宝联合运营订的数据
#         df_group_1 = df.groupby(model).agg({'order_id': 'size', '是否进件': 'sum', '前置拦截': 'sum'})  # , '机审通过件': 'sum', '风控通过件': 'sum'
#         df_group_1.rename(columns={'order_id': '去重订单数', '是否进件': '进件数'}, inplace=True)
#         # 策略241205,策略241212,自有模型回捞策略:2025.8.28：联合拒量订单
#         df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中|联合拒量订单|支付宝联合运营', regex=True)==True]
#         # 剔除支付宝联合运营订单中拒绝理由为空的记录
#         df_241205 = df_241205[~((df_241205.tips.str.contains('支付宝联合运营订单')) & (df_241205['拒绝理由'].str.strip() != ''))]
#         # 拒绝
#         df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机', '云启德曜'])]
#         # 过往机审拒量逻辑
#         # df_241205.loc[:, '机审强拒_拒量'] = np.where(df_241205.qvt_risk=='1', 1, 0)
#         # 跑4月以来的数据可以这样做，跑历史数据还是需要使用旧逻辑qvt_risk=='1'
#         df_241205.loc[:, '机审强拒_拒量'] = np.where(df_241205.order_id.notna(), 1, 0)        
        
#         # df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
#         # 如果订单为联合拒量订单或支付宝联合运营订单，则出库前强拒_拒量为0，即不计入出库前强拒，修改时间：2025-09-22
#         df_241205.loc[:, '出库前强拒_拒量'] = np.where(
#             ((df_241205['qvt_risk'] == '0') | (df_241205['qvt_risk'].isna())) & 
#             (df_241205['qvt_result'] == '1') & 
#             (~df_241205['tips'].str.contains(r'联合拒量订单|支付宝联合运营订单', na=False)), 
#             1, 0)
        
#         # 如果订单为联合拒量订单或支付宝联合运营订单且是否出库列为1，则是否出库置为0，即不计入拒量出库数，修改时间：2025-09-22
#         df_241205.loc[:, '是否出库'] = np.where((df_241205.tips.str.contains(r'联合拒量订单|支付宝联合运营订单', na=False)) & (df_241205['是否出库'] == 1), 
#         0, df_241205['是否出库'])
        
#         df_241205_group = df_241205.groupby(model).agg({'机审强拒_拒量': 'sum', '出库前强拒_拒量': 'sum', '是否出库': 'sum'})
#         df_241205_group.rename(columns={'是否出库': '拒量出库'}, inplace=True)

#         # 不包含拒量数据
#         df_group_2 = df2.groupby(model).agg({'机审强拒': 'sum', '机审通过件': 'sum', '进件前取消': 'sum', '风控通过件': 'sum'
#                 , '人审拒绝': 'sum', '客户取消': 'sum', '已退款': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum'}) # '已退款': 'sum',
#         df_group_2.rename(columns={'是否出库': '出库'}, inplace=True)

#         df_group_3 = pd.merge(df_group_1, df_group_2, on=model, how='inner')
#         # 拼接下拒量的被拒数据
#         df_xmy_all, df_xmy = self.xmy(df, df_risk_examine, model)
#         df_group = df_group_3.merge(df_xmy, on=model, how='left').merge(df_241205_group, on=model, how='left')#.merge(df_241212_group, on=model, how='left')
#         df_group = df_group.fillna(0)
#         df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒'] + df_group['机审强拒_拒量'] #+ df_group['机审通过件_241212']
#         # 总体机审通过件，包含拒量
#         df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']  # +除机审强拒的单
#         df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']  # +出库前强拒的单
#         df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
#         df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']
#         df_group['客户取消'] = df_group['客户取消']+df_group['已退款']

#         df_group["拦截率"] = df_group["前置拦截"] / df_group["去重订单数"]
#         df_group["拦截率"] = df_group["拦截率"].apply(lambda x: format(x, ".2%"))

#         df_group["进件前取消率"] = df_group["进件前取消"] / df_group["去重订单数"]
#         df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x: format(x, ".2%"))

#         df_group['预授权通过率'] = df_group["进件数"] / df_group["去重订单数"]
#         df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x: format(x, ".2%"))

#         df_group["强拒比例"] = df_group["机审强拒"] / df_group["进件数"]
#         df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x: format(x, ".2%"))

#         df_group["风控通过率"] = df_group["风控通过件"] / df_group["进件数"]
#         df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x: format(x, ".2%"))

#         df_group["取消率"] = df_group["客户取消"] / df_group["进件数"]
#         df_group["取消率"] = df_group["取消率"].apply(lambda x: format(x, ".2%"))

#         df_group["无法联系占比"] = df_group["无法联系"] / df_group["进件数"]
#         df_group["无法联系占比"] = df_group["无法联系占比"].apply(lambda x: format(x, ".2%"))

#         df_group["人审拒绝率"] = df_group["人审拒绝"] / df_group["进件数"]
#         df_group["人审拒绝率"] = df_group["人审拒绝率"].apply(lambda x: format(x, ".2%"))

#         df_group["出库前强拒比例"] = df_group["出库前风控强拒"] / df_group["进件数"]
#         df_group["出库前强拒比例"] = df_group["出库前强拒比例"].apply(lambda x: format(x, ".2%"))

#         df_group["进件出库率"] = df_group["出库"] / df_group["进件数"]
#         df_group['总体进件出库率（含拒量）'] = (df_group['出库'] + df_group['拒量出库'])/df_group["进件数"]
#         df_group['拒量进件出库率增加'] = df_group['总体进件出库率（含拒量）'] - df_group['进件出库率']
#         df_group["进件出库率"] = df_group["进件出库率"].apply(lambda x: format(x, ".2%"))
#         df_group["总体进件出库率（含拒量）"] = df_group["总体进件出库率（含拒量）"].apply(lambda x: format(x, ".2%"))
#         df_group["拒量进件出库率增加"] = df_group["拒量进件出库率增加"].apply(lambda x: format(x, ".2%"))

#         df_group["订单出库率"] = df_group["出库"] / df_group["去重订单数"]
#         df_group["订单出库率"] = df_group["订单出库率"].apply(lambda x: format(x, ".2%"))

#         df_group["人审转化率"] = df_group["出库"] / df_group["机审通过件"]
#         df_group["人审转化率"] = df_group["人审转化率"].apply(lambda x: format(x, ".2%"))

#         return df_group

# 合并数据

In [ ]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df_order, df_risk_examine, df_risk, df_ra, df_re = select_data(Time='23:59:59')
df_order.shape
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()
# df, df2 = process_data(df_order, df_risk, df_ra, df_re)

np.int64(173006)

# 处理日期

In [ ]:
# df[df['order_id'] == 3909094]

In [ ]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
print(df['下单日期'].unique())
df["月份"]=df["下单日期"].dt.month
df['date']=df['create_time'].dt.date
df['hour']=df['create_time'].dt.hour
df.shape

<DatetimeArray>
['2025-11-04 00:00:00', '2025-11-05 00:00:00', '2025-11-06 00:00:00',
 '2025-11-07 00:00:00', '2025-11-08 00:00:00', '2025-11-09 00:00:00',
 '2025-11-10 00:00:00', '2025-11-11 00:00:00', '2025-11-12 00:00:00',
 '2025-11-13 00:00:00']
Length: 10, dtype: datetime64[ns]


(173006, 58)

In [ ]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                                           49044
命中蚁盾分3.0强拒                                                                                                 22947
命中冰鉴青云分强拒                                                                                                  17512
命中TD212强拒                                                                                                  12473
命中强拒加入临时黑名单                                                                                                 9150
                                                                                                           ...  
命中冰鉴青云分强拒, 命中策略JDB002_212强拒, 命中景行分DE303强拒, 命中极瞳分V2强拒, 命中极瞳分V3强拒, 评分等级低于C,命中拒绝                                  1
命中风险等级联合规则强拒, 命中策略240703_14强拒, 命中策略240703_4强拒                                                                  1
命中JDB002_201048强拒, 命中景行分DE303强拒, 命中极瞳分V2组合规则强拒, 命中青云分25v1强拒, 命中极瞳分V2强拒, 评分等级低于C,命中拒绝       

# 处理备注信息

In [ ]:
# df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()

In [ ]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
# df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
# df["内存"].value_counts() 

def getfangan(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="租赁方案":
            return color_list[j]["value"]
df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)
# df["租赁方案"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
# df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['机型'] + df['内存']
# 当order_id相同时，将所有记录的total_describes 进行拼接成一个字符串并赋值给total_describes
# 第一步：当 order_id 相同时，将所有记录的 total_describes 拼接成一个字符串并赋值给备注
# 第一步：拼接 total_describes
df['total_describes'] = df['total_describes'].fillna('')
df.loc[:, '备注'] = df.groupby('order_id')['total_describes'].transform(
    lambda x: ','.join(x)
)

# 第二步：提取并重新拼接 msg 内容
df.loc[:, '备注'] = df.groupby('order_id')['备注'].transform(
    lambda x: ','.join(
        [msg for text in x for msg in 
        __import__('re').findall(r'msg=(.*?)"', text) if msg]
    )
)
df[df['order_number']=='A20251006182029112']['备注']

Series([], Name: 备注, dtype: object)

In [ ]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    168259
1      4747
Name: count, dtype: int64

# 渠道归属

In [ ]:
df.loc[:,"归属渠道"]=df.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"],x['channel_type_id'],x['order_type']),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                   110222
搜索渠道                    29805
京东渠道                    20216
单人聊天会话中的小程序消息卡片（分享）      4214
八派信息                     2343
                        ...  
京东出游季                       1
卡包\r\n                      1
搜索侠客行-首页                    1
搜索-租苹果手机                    1
租机宝                         1
Name: count, Length: 44, dtype: int64

# 订单去重

In [ ]:
# 保留不去重的数据
df_not_drop = df.copy()
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


# df.drop(df[df['true_name'].isin(
#             ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
#                 , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)
df.shape

(76687, 77)

# 定义状态

In [ ]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [ ]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()

审核状态
前置拦截       27372
进件前取消      23018
机审强拒       12352
未进件         6136
人审拒绝        4049
出库          2033
已退款          689
无法联系         564
待审核          238
出库前风控强拒      159
客户取消          74
检测中            3
Name: count, dtype: int64

# 保留商家的数据

In [ ]:
df_contain = df.copy()

# 剔除商家数据

In [ ]:
df = clean.drop_merchant(df)

# 去重订单明细导出

In [ ]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-11-13'

# 导入服务订单

In [ ]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(18945, 86)

In [ ]:
df = clean.status_node(df)

# 导入台账数据

In [ ]:
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")  
df_ck.shape

(43258, 18)

In [ ]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)


In [ ]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

# 剔除拒量订单

In [ ]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

# 临时需求

## 租归转租送订单

In [ ]:
# tod.change_sku_id > 0且不为空
sql_rent_to_send = '''
select
order_id, change_sku_id
from db_digua_business.t_order_details tod
where change_sku_id is not null
and change_sku_id > 0
'''
df_change = clean.query(sql_rent_to_send)
df_iphone17 = df[(df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))]# &(df2['下单日期']=='2025-11-03')
df_iphone17_all = df_iphone17.merge(df_change, on='order_id', how='left')


In [ ]:
df_iphone17_all[(df_iphone17_all['change_sku_id'].notna())&(df_iphone17_all['下单日期']=='2025-11-10')&(df_iphone17_all['是否出库']==1)][['order_number', 'change_sku_id','租赁方案','归属渠道']]#.shape

,order_number,change_sku_id,租赁方案,归属渠道
12346,A2025111008244390,13544.0,租完即送,京东渠道
12358,A202511101055371031,13510.0,租完即送,京东渠道
12369,A20251110135355952,13457.0,租完即送,京东渠道
12376,A20251110144831979,13454.0,租完即送,京东渠道
12377,A20251110150517139,13456.0,租完即送,京东渠道
...,...,...,...,...
12417,A20251110195958820,13501.0,租完即送,顶部搜索框的搜索结果页
12419,A20251110224457355,13503.0,租完即送,京东渠道
12420,A2025111023081144,13558.0,租完即送,京东渠道
12426,A20251110123224533,13465.0,租完即送,京东渠道


In [ ]:
df_iphone17_all[(df_iphone17_all['下单日期']=='2025-11-09')&(df_iphone17_all['是否出库']==1)&(df_iphone17_all['租赁方案']=='租完即送')][['order_number', 'change_sku_id','租赁方案','归属渠道']]#.shape#[[]]

,order_number,change_sku_id,租赁方案,归属渠道
10090,A20251109160850195,13436.0,租完即送,京东渠道
10092,A202511091658301163,13437.0,租完即送,京东渠道
10096,A20251109201130149,13444.0,租完即送,京东渠道
10101,A20251109094724322,13423.0,租完即送,京东渠道
10103,A20251109103233375,13424.0,租完即送,京东渠道
10108,A20251109112619374,NaN,租完即送,我的小程序入口
10129,A2025110919033556,13442.0,租完即送,京东渠道
10132,A20251109174609801,13440.0,租完即送,京东渠道


## 配对信审跟进人（上一天的数据，10点）

In [ ]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = query(sql_name)# df_iphone17[df_iphone17['order_id'] == 3909094]

In [ ]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
df_name = df_name[df_name.信审员.isin(name_list)]

In [ ]:
# df_name[df_name.order_id==4367216]

In [ ]:
df_iphone17 = df2[(df2['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))]# &(df2['下单日期']=='2025-11-03')
df_iphone17_all = df_iphone17.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
df_iphone17_all[df_iphone17_all['机审通过件']==1][['order_number', '信审员', '电审拒绝原因', '备注']].to_excel('F:\需求\总需求\iPhone17系列人审拒绝原因备注_11.07.xlsx', index=False)

In [ ]:
# df_iphone17_all[df_iphone17_all['待审核'] == 1][['下单日期', 'order_number', '机型内存','归属渠道', '审核状态', '颜色', '信审员']].to_excel('F:\需求\瑞彤需求\待审核匹配信审人订单.xlsx', index=False)

df_iphone17 = df[(df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))&(df['下单日期']=='2025-10-27')&(df['待审核']==1)]# 
df_iphone17[['order_number', 'product_name', '机型内存','归属渠道', 'status2', '电审拒绝原因', '备注']]

,order_number,product_name,机型内存,归属渠道,status2,电审拒绝原因,备注


## 17PM512G租送套餐每天机审通过和出库前強拒通过订单数，9.20之后

In [ ]:
df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')].product_name.unique()

array(['首月1元 iPhone 17 Pro 无溢价全新苹果 非监管机 顺丰包邮 ',
       '无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮',
       '首月1元 iPhone 17 全新无锁 非监管机 顺丰包邮', 'iPhone 17 全新苹果新品 非监管机 顺丰包邮【社群】',
       '首月1元 iPhone 17 Pro Max 非监管机 顺丰包邮 【活动专享】',
       '立减1111元 iPhone 17Pro无溢价非监管机 顺丰包邮【双11限时活动',
       '首月1元 iPhone 17 全新无锁 非监管机 顺丰包邮【活动专享】',
       'iPhone 17 Pro Max 全新 国行正品 顺丰包邮 按下单顺序陆续发货',
       '【立减1111】iPhone 17 Pro 全新无锁 非监管机 顺丰包邮',
       '首月1元 iPhone 17 Pro Max 无锁 非监管机 顺丰包邮 【商机】',
       '【首月1元】iPhone17ProMax 全新无锁 无溢价 非监管机 顺丰包邮',
       '首月1元 iPhone 17 Pro Max 全新无溢价无锁 非监管机 顺丰包邮',
       '首月1元 iPhone17Pro 全新无锁 非监管机 顺丰包邮【活动专享】',
       '首月1元 iPhone 17 Pro Max 非激活 非监管机 顺丰包邮 【租物】',
       '【首月5折】iPhone 17 全新 非监管机 顺丰包邮', '首月1元iPhone 17 Pro Max  顺丰包邮 非监管机',
       '首月1元 iPhone Air 全新无锁 非监管机 顺丰包邮',
       'iPhone 17ProMax 全新苹果新品 非监管机顺丰包邮【直播',
       '首月1元 iPhone 17 Pro 全新非激活 非监管机 顺丰包邮 【租物】',
       '首月5折 iPhone Air 全新 非监管机 顺丰包邮 按下单顺序陆续发货',
       'iPhone 17 Pro 首月1元 官方正品 顺丰包邮 无溢价无监管',
       '首月1元 iPhone 17 Pro

In [ ]:
df_iphone17PM = df[(df['机型']=='iPhone 17 Pro Max') & (df['租赁方案']=='租完即送')]
# 按下单日期与身份证号去重
df_iphone17PM.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)
df_iphone17PM.loc[:, '出库前強拒通过件'] = np.where((df_iphone17PM['出库前风控强拒']==0)&(df_iphone17PM['风控通过件']==1), 1, 0)
df_iphone17PM_group = df_iphone17PM.groupby(['下单日期']).agg({'机审通过件':'sum','风控通过件':'sum','客户取消':'sum',"无法联系":'sum','出库前风控强拒':'sum','出库前強拒通过件':'sum', '是否出库':'sum'})
df_iphone17PM_group.loc['汇总', :] = df_iphone17PM_group.select_dtypes(include=[np.number]).sum()
df_iphone17PM_group

,机审通过件,风控通过件,客户取消,无法联系,出库前风控强拒,出库前強拒通过件,是否出库
下单日期,,,,,,,
2025-11-04 00:00:00,18.0,6.0,0.0,1.0,0.0,6.0,5.0
2025-11-05 00:00:00,23.0,9.0,0.0,2.0,0.0,9.0,5.0
2025-11-06 00:00:00,12.0,6.0,0.0,1.0,0.0,6.0,4.0
2025-11-07 00:00:00,15.0,9.0,0.0,2.0,2.0,7.0,5.0
2025-11-08 00:00:00,19.0,10.0,0.0,0.0,3.0,7.0,5.0
2025-11-09 00:00:00,22.0,12.0,0.0,1.0,0.0,12.0,7.0
2025-11-10 00:00:00,54.0,31.0,0.0,1.0,1.0,30.0,27.0
2025-11-11 00:00:00,63.0,32.0,1.0,4.0,1.0,31.0,23.0
2025-11-12 00:00:00,66.0,41.0,0.0,1.0,0.0,41.0,34.0


In [ ]:
df_iphone17PM['客户取消']=np.where(df_iphone17PM['status2']=='已退款', 1, df_iphone17PM['客户取消'])
df_iphone17PM[df_iphone17PM['出库前強拒通过件']==1][['order_number', 'order_id', '租赁方案', '机审通过件', '客户取消',"无法联系", '出库前风控强拒', '出库前強拒通过件', '是否出库','status2']]

,order_number,order_id,租赁方案,机审通过件,客户取消,无法联系,出库前风控强拒,出库前強拒通过件,是否出库,status2
7575,A20251104234139262,4454260,租完即送,1,1,1,0,1,0,已退款
7591,A2025110408052520,4442641,租完即送,1,0,0,0,1,1,租赁中
7639,A20251104113052478,4444740,租完即送,1,0,0,0,1,1,租赁中
7640,A20251104113140487,4444749,租完即送,1,0,0,0,1,1,租赁中
7730,A20251104161004179,4449098,租完即送,1,0,0,0,1,1,租赁中
...,...,...,...,...,...,...,...,...,...,...
76657,A20251113140732117,4573817,租完即送,1,0,0,0,1,1,待发货
76660,A20251113141542244,4573929,租完即送,1,0,0,0,1,1,待发货
76676,A20251113144943863,4574535,租完即送,1,0,0,0,1,1,待发货
76677,A2025111315032273,4574786,租完即送,1,0,0,0,1,1,待发货


##  iPhone17系列人审拒绝原因汇总与明细

In [ ]:
# df_iphone17[['order_id', 'total_describes']].value_counts()

In [ ]:
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
dfds = df_iphone17[df_iphone17['人审拒绝']==1][['order_number', 'status2', '归属渠道', '电审拒绝原因', '备注']]
dfds

,order_number,status2,归属渠道,电审拒绝原因,备注
5579,A2025110400092247,已退款,小程序商家消息（服务提醒）,"命中高风险渠道""]","高风险,审核不通过：命中高风险渠道"
5614,A20251104005358290,已退款,搜索渠道,"拒绝电审""]","不要了,审核不通过：拒绝电审"
5621,A2025110401060429,已退款,芝麻租物,"已在其他平台租赁""]","未接,1+2+碎屏,不接受预收租金,审核不通过：已在其他平台租赁"
5634,A20251104012935159,已退款,芝麻租物,"命中高风险渠道""]","命中高风险渠道,审核不通过：命中高风险渠道"
5639,A20251104013922206,已退款,支付宝客户端首页,"拒绝电审""]","烧烤店工作，1年,待付1+碎屏险，未核实联系人,对活动理解错误,审核不通过：拒绝电审"
...,...,...,...,...,...
76344,A202511131558171181,已退款,支付宝客户端首页,"拒绝电审""]","不接受换租送,审核不通过：拒绝电审"
76379,A20251113161742307,已退款,单人聊天会话中的小程序消息卡片（分享）,"命中高风险地址""]","高风险地址,审核不通过：命中高风险地址"
76388,A20251113162051366,已退款,京东渠道,"命中高风险地址""]","命中高风险地址,审核不通过：命中高风险地址"
76390,A20251113162245405,已退款,搜索渠道,"短期在租多台""]","近期有在租同型号,审核不通过：短期在租多台"


In [ ]:
# dfds_g = dfds.groupby('电审拒绝原因').agg({'order_number':'count'}).rename(columns={'order_number':'数量'}).sort_values('数量',ascending=False)
# dfds_g.loc[:,'占比'] = (dfds_g['数量']/dfds_g['数量'].sum()).apply(lambda x:format(x,".2%"))
# dfds_g
# with pd.ExcelWriter('F:\需求\瑞彤需求\iPhone17系列人审拒绝原因备注汇总10.23.xlsx') as writer:
#     dfds_g.to_excel(writer, sheet_name='汇总', index=True)
#     dfds.to_excel(writer, sheet_name='明细', index=False)

# iPhone 17 全系列

## iPhone17系列总体数据

In [ ]:
# df_iphone17[(df_iphone17.下单日期=='2025-10-27')&(df_iphone17['是否出库']==1)][['order_number', 'status2', '归属渠道','merchant_name', 'product_name',]]
# df[(df.order_number == 'A20251027094406258')][['order_number', 'product_name', '机型']]# &(df['product_name'].str.contains(r'iPhone 17|iPhone Air'))
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
df_iphone17[(df_iphone17['下单日期']=='2025-10-28')&(df_iphone17['是否出库']==1)][['order_number', 'status2', '归属渠道','merchant_name', 'product_name','机型']]

,order_number,status2,归属渠道,merchant_name,product_name,机型


In [ ]:
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_all = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, '下单日期')
df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
df_iphone17_all_new = df_iphone17_all.reset_index()
df_iphone17_all_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-11-04 00:00:00,1272.0,372.0,29.25%,285.0,22.41%,206.0,72.28%,79.0,45.0,34.0,11.93%,5.0,5.0,5.0,0.0,19.0,0.0,6.67%,6.67%,0.00%,1.75%,15.79%,1.75%,1.75%,1.49%
1,2025-11-05 00:00:00,1197.0,310.0,25.90%,267.0,22.31%,185.0,69.29%,82.0,43.0,39.0,14.61%,6.0,4.0,10.0,0.0,19.0,0.0,7.12%,7.12%,0.00%,2.25%,16.10%,3.75%,1.50%,1.59%
2,2025-11-06 00:00:00,1280.0,331.0,25.86%,286.0,22.34%,202.0,70.63%,84.0,50.0,34.0,11.89%,7.0,7.0,3.0,0.0,17.0,0.0,5.94%,5.94%,0.00%,2.45%,17.48%,1.05%,2.45%,1.33%
3,2025-11-07 00:00:00,1346.0,371.0,27.56%,277.0,20.58%,192.0,69.31%,85.0,31.0,54.0,19.49%,6.0,13.0,10.0,0.0,25.0,0.0,9.03%,9.03%,0.00%,2.17%,11.19%,3.61%,4.69%,1.86%
4,2025-11-08 00:00:00,1359.0,349.0,25.68%,300.0,22.08%,215.0,71.67%,85.0,30.0,55.0,18.33%,9.0,11.0,6.0,0.0,29.0,0.0,9.67%,9.67%,0.00%,3.00%,10.00%,2.00%,3.67%,2.13%
5,2025-11-09 00:00:00,1339.0,378.0,28.23%,296.0,22.11%,208.0,70.27%,88.0,43.0,45.0,15.20%,7.0,10.0,3.0,0.0,25.0,0.0,8.45%,8.45%,0.00%,2.36%,14.53%,1.01%,3.38%,1.87%
6,2025-11-10 00:00:00,1681.0,471.0,28.02%,416.0,24.75%,279.0,67.07%,137.0,70.0,67.0,16.11%,15.0,6.0,5.0,0.0,41.0,0.0,9.86%,9.86%,0.00%,3.61%,16.83%,1.20%,1.44%,2.44%
7,2025-11-11 00:00:00,1901.0,480.0,25.25%,445.0,23.41%,291.0,65.39%,154.0,66.0,88.0,19.78%,15.0,8.0,4.0,3.0,58.0,0.0,13.03%,13.03%,0.00%,3.37%,14.83%,0.90%,1.80%,3.05%
8,2025-11-12 00:00:00,2197.0,587.0,26.72%,464.0,21.12%,333.0,71.77%,131.0,45.0,86.0,18.53%,13.0,5.0,2.0,8.0,58.0,0.0,12.50%,12.50%,0.00%,2.80%,9.70%,0.43%,1.08%,2.64%
9,2025-11-13 00:00:00,1325.0,369.0,27.85%,291.0,21.96%,168.0,57.73%,123.0,45.0,78.0,26.80%,6.0,2.0,6.0,34.0,30.0,0.0,10.31%,10.31%,0.00%,2.06%,15.46%,2.06%,0.69%,2.26%


In [ ]:
# df_iphone17[(df_iphone17['是否进件']==1)&(df_iphone17['下单日期']=='2025-09-25')]['order_number'].to_excel('F:\需求\总需求\iPhone17系列订单.xlsx')
df_iphone17[(df_iphone17['下单日期']=='2025-10-20')&(df_iphone17['待审核']==1)][['order_id', 'order_number', 'status','是否拒量']]

,order_id,order_number,status,是否拒量


## iPhone17系列租物数据

In [ ]:
# df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
# df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
# df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
# df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
# df_iphone17_zw_group_new

## iPhone17系列搜索数据

In [ ]:
# df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
# df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
# df_iphone17_ss_group = all_models.data_group(df_iphone17_ss, df_iphone17_ss_2, df_risk_examine, '下单日期')
# df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
# df_iphone17_ss_group_new

## iPhone17系列京东数据

In [ ]:
df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
# df_iphone17_jd_group = all_models.data_group(df_iphone17_jd, df_iphone17_jd_2, df_risk_examine, '下单日期')
# df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
# df_iphone17_jd_group_new.tail()
df_iphone17_jd.下单日期.unique()

<DatetimeArray>
['2025-11-04 00:00:00', '2025-11-05 00:00:00', '2025-11-06 00:00:00',
 '2025-11-07 00:00:00', '2025-11-08 00:00:00', '2025-11-09 00:00:00',
 '2025-11-10 00:00:00', '2025-11-11 00:00:00', '2025-11-12 00:00:00',
 '2025-11-13 00:00:00']
Length: 10, dtype: datetime64[ns]

### iPhone17系列京东机型转化数据

In [ ]:
df_iphone17_jd_1020 = df_iphone17_jd[df_iphone17_jd['下单日期']=='2025-10-20']
df_iphone17_jd_2_1020 = df_iphone17_jd_2[df_iphone17_jd_2['下单日期']=='2025-10-20']
df_iphone17_jd_group = all_models.data_group(df_iphone17_jd_1020, df_iphone17_jd_2_1020, df_risk_examine, '机型内存')
df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
df_iphone17_jd_group_new#.tail()

,机型内存,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,合计,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_iphone17_jd_1021 = df_iphone17_jd[df_iphone17_jd['下单日期']=='2025-10-21']
df_iphone17_jd_2_1021 = df_iphone17_jd_2[df_iphone17_jd_2['下单日期']=='2025-10-21']
df_iphone17_jd_group_1021 = all_models.data_group(df_iphone17_jd_1021, df_iphone17_jd_2_1021, df_risk_examine, '机型内存')
df_iphone17_jd_group_1021 = df_iphone17_jd_group_1021[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_jd_group_1021.loc['合计'] = df_iphone17_jd_group_1021.select_dtypes(include=[np.number]).sum()
df_iphone17_jd_group_1021_new = df_iphone17_jd_group_1021.reset_index()
df_iphone17_jd_group_1021_new#.tail()

,机型内存,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,合计,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# with pd.ExcelWriter('F:\需求\崔婷玉需求\iPhone17系列京东机型转化数据.xlsx') as writer:
#     df_iphone17_jd_group_new.to_excel(writer, sheet_name='10月20日', index=False)
#     df_iphone17_jd_group_1021_new.to_excel(writer, sheet_name='10月21日', index=False)

In [ ]:
# # 找出重复的订单号
# duplicated_order_numbers = df_iphone17_jd[df_iphone17_jd.order_number.duplicated(keep=False)].order_number

# # 输出重复的订单号（去重显示）
# print(duplicated_order_numbers.unique())


# # df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17_2['已退款']==1)]

## iPhone17系列客户取消数据

In [ ]:
# df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
# df_iphone17_qx

## iPhone17系列无法联系数据

In [ ]:
# df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
# df_iphone17_wf


# iPhone 16 的遗老遗少可能有用

In [ ]:

# df_16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_16 = df_16[df_16['product_name']!='iPhone 16 2024年新机 200元权益包']
# df_16['product_name'] = df_16['product_name'].str.replace(' ', '', regex=False)
# # df_16['型号内存'] = df_16['product_name'].str.split(' 国').str.get(0).str.split(' 2').str.get(0).str.replace(' ', '', regex=False).str.replace('首月5折', '', regex=False)
# df_16['型号内存'] = df_16['product_name'].str.extract(r'(iPhone ?16(ProMax|Pro|Plus)?)', expand=True)[0]

# def iPhone(df, qd_names):
#     # df_16_group = df_16.groupby(['归属渠道','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
#     df_16_group = df.groupby([f'{qd_names}','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})

#     df_16_group.rename(columns={'order_id':'去重订单数'},inplace=True )

#     df_16_group["拦截率"] = df_16_group["前置拦截"]/df_16_group["去重订单数"]
#     df_16_group["拦截率"] = df_16_group["拦截率"].apply(lambda x:format(x,".2%"))

#     df_16_group['预授权通过率'] = df_16_group["是否进件"]/df_16_group["去重订单数"]
#     df_16_group["预授权通过率"] = df_16_group["预授权通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["强拒比例"] = df_16_group["机审强拒"]/df_16_group["是否进件"]
#     df_16_group["强拒比例"] = df_16_group["强拒比例"].apply(lambda x:format(x,".2%"))

#     df_16_group["机审通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]

#     df_16_group["风控通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]-df_16_group["人审拒绝"]
#     df_16_group["风控通过率"] = df_16_group["风控通过件"]/df_16_group["是否进件"]
#     df_16_group["风控通过率"] = df_16_group["风控通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["取消率"] = df_16_group["客户取消"]/df_16_group["是否进件"]
#     df_16_group["取消率"] = df_16_group["取消率"].apply(lambda x:format(x,".2%"))


#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝"]/df_16_group["是否进件"]
#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

#     df_16_group["进件出库率"]=df_16_group["是否出库"]/df_16_group["是否进件"]
#     df_16_group["进件出库率"]=df_16_group["进件出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group["订单出库率"]=df_16_group["是否出库"]/df_16_group["去重订单数"]
#     df_16_group["订单出库率"]=df_16_group["订单出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group = df_16_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核"]]
#     df_16_group
#     return df_16_group


In [ ]:
# # 计算环比
# def calculate_growth_rate(current, previous):
#     if previous == 0:
#         return float('inf') if current > 0 else float('-inf')
#     return (current - previous) / previous

In [ ]:
# # 获取渠道的具体渠道
# qd_names = ['来源渠道', '归属渠道']
# indx1 = 1
# qd_group = iPhone(df_16, qd_names[indx1])
# qd_lists = ['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']
# qd_list =  qd_group.reset_index()[qd_names[indx1]][qd_group.reset_index()[qd_names[indx1]].isin(qd_lists)].drop_duplicates().to_list()


In [ ]:
# # 渠道明细 去重订单数
# qd_type = ['去重订单数', '是否进件']
# indx2 = 0
# df_16_group_qcdd = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_qcdd = df_16_group_qcdd.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_qcdd = df_16_group_qcdd.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_qcdd = df_pivot_qcdd[ qd_list].reset_index()

In [ ]:
# # 渠道明细 是否进件
# indx2 = 1
# df_16_group_jj = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_jj = df_16_group_jj.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_jj = df_16_group_jj.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_jj = df_pivot_jj[ qd_list].reset_index()

In [ ]:
# # 渠道数据汇总
# # 提取最后两行的下单日期用于对比
# current = df_pivot_jj['下单日期'].iloc[-1].strftime('%m%d')
# previous = df_pivot_jj['下单日期'].iloc[-2].strftime('%m%d')
# df_16_qcdd = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# df_16_jj = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# for qd_name in qd_list:
#     df_16_qcdd[qd_name] =  [df_pivot_qcdd[qd_name].diff().iloc[-1],  (df_pivot_qcdd[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]
#     df_16_jj[qd_name] =  [df_pivot_jj[qd_name].diff().iloc[-1],  (df_pivot_jj[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]

# indx = 0
# df_pivot_qcdds = pd.concat([df_pivot_qcdd[indx:], df_16_qcdd])
# df_pivot_jjs = pd.concat([df_pivot_jj[indx:], df_16_jj])
# # df_pivot_jjs

# 设置定时任务

In [9]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc

# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1(Time):
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的{Time}')
    
    df_order, df_risk_examine, df_risk, df_ra, df_re = select_data(Time)
    # df_risk_examine = query(sql3)
    # 数据处理
    df, df2 = process_data(df_order, df_risk, df_ra, df_re)
    
    #iphone 17系列转化数据，总体
    df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
    # df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
    df_iphone17_all = group_data(df_iphone17, '下单日期')
    df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","拒量订单数","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    # 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
    df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
    df_iphone17_all_new = df_iphone17_all.reset_index()
    
    # 租物
    df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
    # df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
    # df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
    df_iphone17_zw_group = group_data(df_iphone17_zw, '下单日期')
    df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","拒量订单数","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
    
    # 搜索渠道
    df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
    # df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
    df_iphone17_ss_group = group_data(df_iphone17_ss, '下单日期')
    df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","拒量订单数","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
    
    # 京东渠道
    df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
    # df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
    df_iphone17_jd_group = group_data(df_iphone17_jd, '下单日期')
    df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","拒量订单数","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
    
    # 客户取消数据
    df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]

    # 无法联系数据
    df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
    
    
    with pd.ExcelWriter(path1 + f'iPhone 17系列_{Today1}.xlsx', engine='openpyxl') as writer:
        df_iphone17_all_new.to_excel(writer, sheet_name='iPhone17系列总体', index=False)
        df_iphone17_zw_group_new.to_excel(writer, sheet_name='iPhone17系列芝麻租物', index=False)
        df_iphone17_ss_group_new.to_excel(writer, sheet_name='iPhone17系列搜索渠道', index=False)
        df_iphone17_jd_group_new.to_excel(writer, sheet_name='iPhone17系列京东渠道', index=False)
        df_iphone17_qx.to_excel(writer, sheet_name='iPhone17系列取消明细', index=False)
        df_iphone17_wf.to_excel(writer, sheet_name='iPhone17系列无法联系明细', index=False)
    del df_iphone17_all_new,df_iphone17_zw_group_new,df_iphone17_ss_group_new,df_iphone17_jd_group_new,df_iphone17_qx,df_iphone17_wf
    gc.collect()
    print("iPhone 17系列数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler1 = BackgroundScheduler()
scheduler2 = BackgroundScheduler()
# 添加一个cron任务，每天的17点30分执行
hour = 17
minute = 30
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute, args=['17:30:00'])
scheduler2.add_job(my_job1, 'cron', hour=14, minute=1, args=['14:00:00'])
# scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
# my_job1(Time='17:30:00')
# my_job1(Time='14:00:00')
# my_job1(Time='23:59:59')
# 启动scheduler1
scheduler1.start()
scheduler2.start()


## iPhone 17系列昨日信审详情

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc

# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job2(Time):
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的{Time}')
    
    df_order, df_risk_examine = select_data(Time)
    # df_risk_examine = query(sql3)
    # 数据处理
    df, df2 = process_data(df_order)
    
    #iphone 17系列转化数据，总体
    df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
    
    sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
    df_name = clean.query(sql_name)# df_iphone17[df_iphone17['order_id'] == 3909094]
    
    df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
    # 创建分配人映射关系 
    name_dict = {
            '小张': '李巧玲',
            '小周': '李巧凤',
            '小南': '刘三妹',
            '小何': '何静',
            '小谢': '谢金凤',
            '小咪': '杨健',
            '小慧': '林思慧',
            '小滢': '胡彩滢',
            '小星': '廖丽敏',
            '小兰': '黄兰娟',
            '小晚': '周莹',
            '小芳': '罗芳'
            }
    name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

    # 匹配映射字典
    df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
    df_name = df_name[df_name.信审员.isin(name_list)]
    
    df_iphone17 = df2[(df2['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))&(df2['下单日期']=='2025-10-28')]# 
    df_iphone17_all = df_iphone17.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
    
    
    
    
    
        
    
    with pd.ExcelWriter(path1 + f'iPhone17系列人审拒绝原因备注.xlsx_{Today1}.xlsx', engine='openpyxl') as writer:
        df_iphone17_all[df_iphone17_all['机审通过件']==1][['order_number', '信审员', '电审拒绝原因', '备注']].to_excel(writer, sheet_name='人审取消原因备注',index=False)
    del df_iphone17_all
    gc.collect()
    print("iPhone 17系列数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler3 = BackgroundScheduler()
# 添加一个cron任务，每天的17点30分执行
hour = 9
minute = 59
scheduler3.add_job(my_job1, 'cron', hour=hour, minute=minute, args=['23:59:59'])
# scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)  
# my_job1(Time='17:30:00')
# my_job1(Time='14:00:00')
# 启动scheduler1
scheduler3.start()



In [63]:
df_order, df_risk_examine = select_data(Time='17:30:00')

ValueError: too many values to unpack (expected 2)

## iPhone 16系列

In [ ]:

# import gc
# hour = 16
# minute = 30
# # H = datetime.now().strftime('%H')
# # M = datetime.now().strftime('%M')
# def my_job2():
    
#     # print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
#     Today1 = str(datetime.now().strftime('%Y%m%d%H'))
#     print(f'执行定时任务：现在是{Today1}的{hour}点{minute}分')
    
#     df_order = query(sql1)
#     df_risk_examine = query(sql3)
#     # 数据处理
#     df, df2 = process_data(df_order)
    
#     #iphone 17系列转化数据，总体
#     df_iphone16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
#     # df_iphone16_2 = df2[df2['product_name'].str.contains(r'iPhone 16|iPhone Air')]
#     df_iphone16_all = group_data(df_iphone16, '下单日期')
#     df_iphone16_all = df_iphone16_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     # 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
#     df_iphone16_all.loc['合计'] = df_iphone16_all.select_dtypes(include=[np.number]).sum()
#     df_iphone16_all_new = df_iphone16_all.reset_index()
    
#     # 租物
#     df_iphone16_zw = df_iphone16[df_iphone16['归属渠道']=='芝麻租物']
#     # df_iphone16_zw_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='芝麻租物']
#     # df_iphone16_zw_group = all_models.data_group(df_iphone16_zw, df_iphone16_zw_2, df_risk_examine, '下单日期')
#     df_iphone16_zw_group = group_data(df_iphone16_zw, '下单日期')
#     df_iphone16_zw_group = df_iphone16_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_zw_group.loc['合计'] = df_iphone16_zw_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_zw_group_new = df_iphone16_zw_group.reset_index()
    
#     # 搜索渠道
#     df_iphone16_ss = df_iphone16[df_iphone16['归属渠道']=='搜索渠道']
#     # df_iphone16_ss_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='搜索渠道']
#     df_iphone16_ss_group = group_data(df_iphone16_ss, '下单日期')
#     df_iphone16_ss_group = df_iphone16_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_ss_group.loc['合计'] = df_iphone16_ss_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_ss_group_new = df_iphone16_ss_group.reset_index()
    
#     # 京东渠道
#     df_iphone16_jd = df_iphone16[df_iphone16['归属渠道']=='京东渠道']
#     # df_iphone16_jd_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='京东渠道']
#     df_iphone16_jd_group = group_data(df_iphone16_jd, '下单日期')
#     df_iphone16_jd_group = df_iphone16_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_jd_group.loc['合计'] = df_iphone16_jd_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_jd_group_new = df_iphone16_jd_group.reset_index()
    
#     # 客户取消数据
#     df_iphone16_qx = df_iphone16[(df_iphone16['客户取消']==1)|(df_iphone16['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]

#     # 无法联系数据
#     df_iphone16_wf = df_iphone16[(df_iphone16['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
    
    
#     with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl') as writer:
#         df_iphone16_all_new.to_excel(writer, sheet_name='iPhone16系列总体', index=False)
#         df_iphone16_zw_group_new.to_excel(writer, sheet_name='iPhone16系列芝麻租物', index=False)
#         df_iphone16_ss_group_new.to_excel(writer, sheet_name='iPhone16系列搜索渠道', index=False)
#         df_iphone16_jd_group_new.to_excel(writer, sheet_name='iPhone16系列京东渠道', index=False)
#         df_iphone16_qx.to_excel(writer, sheet_name='iPhone16系列取消明细', index=False)
#         df_iphone16_wf.to_excel(writer, sheet_name='iPhone16系列无法联系明细', index=False)
#     del df_iphone16_all_new,df_iphone16_zw_group_new,df_iphone16_ss_group_new,df_iphone16_jd_group_new,df_iphone16_qx,df_iphone16_wf
#     gc.collect()
#     print("iPhone 16系列数据已保存,内存已回收")
#     # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
#     #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

# # scheduler1 = BackgroundScheduler()
# # # 添加一个cron任务，每天的16点30分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# # scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
# my_job2()
# # 启动scheduler1



执行定时任务：现在是2025102709的16点30分
<DatetimeArray>
['2025-10-21 00:00:00', '2025-10-22 00:00:00', '2025-10-23 00:00:00',
 '2025-10-24 00:00:00', '2025-10-25 00:00:00', '2025-10-26 00:00:00',
 '2025-10-27 00:00:00']
Length: 7, dtype: datetime64[ns]
iPhone 16系列数据已保存,内存已回收


In [ ]:
# df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
# df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
# # 筛选支付宝和京东的数据
# df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
# df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
# df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
# df_iphone17_group = df_iphone17_group[['进件数', '出库']]
# # 计算进件占比与出库占比
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group

In [ ]:
# # iPhone16系列、iPhone17系列占比情况更新
# # 机型+内存+渠道（支付宝、京东）+进件+出库+进件占比+出库占比，
# # 每日数据更新 至17预租数据文件内。
# def myjob2():
#     minute = 35
#     path = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
#     Today = str(datetime.now().strftime('%Y%m%d%H'))
#     print(f'执行定时任务：现在是{Today}的{minute}分')
    
#     df_order = query(sql1)
#     df_risk_examine = query(sql3)
#     # 数据处理
#     df, df2 = process_data(df_order)
#     df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
#     df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
#     # 筛选支付宝和京东的数据
#     df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
#     df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
    
#     # 筛选iphone16系列数据
#     df_iphone16 = df[df.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_2 = df2[df2.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_group = all_models.data_group(df_iphone16, df_iphone16_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone16_group = df_iphone16_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件数'] / df_iphone16_group['进件数'].sum()
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库'] / df_iphone16_group['出库'].sum()
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     #iphone 17系列转化数据，总体
#     df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone17_group = df_iphone17_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     with pd.ExcelWriter(path + f'iPhone 16-17系列占比_{Today}.xlsx', engine='openpyxl') as writer:
#         df_iphone16_group.to_excel(writer, sheet_name='iPhone16系列占比', index=True)
#         df_iphone17_group.to_excel(writer, sheet_name='iPhone17系列占比', index=True)
        
#     del df_iphone16_group,df_iphone17_group
#     gc.collect()
#     print("iPhone 16、17系列占比数据已保存,内存已回收")


# myjob2()
# scheduler2 = BackgroundScheduler()
# # 添加一个cron任务，每天的17点35分执行
# scheduler2.add_job(myjob2, 'cron', hour=17, minute=35)

# # 启动scheduler1
# scheduler2.start()


In [ ]:
# ip16 = pd.read_excel(r'\\digua\迪瓜租机\18.iPhone16预租数据/iPhone 16系列_2024092018.xlsx')
# ip16 = ip16.ffill()
# ip16['下单日期2'] = pd.to_datetime(ip16['下单日期']).dt.date
# ip16_group = ip16.groupby(['来源渠道', '下单日期2']).agg({'去重订单数': 'sum', '是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['来源渠道', '下单日期2'], ascending=(False, False))
# # ip16_group = ip16_group.sort_values(by=['下单日期2'], ascending=(False))
# # with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
# #         ip16_group.to_excel(writer, sheet_name='iPhone16系列转化')
# ip16_group

In [ ]:

# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'去重订单数': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='去重订单数')
# df_pivot1 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
#         df_pivot1.to_excel(writer, sheet_name='iPhone16系列转化(去重订单数)')
# # ip16_group

In [ ]:
# ip16['去重订单数'].sum()

In [ ]:
# 
# ip16_group['去重订单数'].sum()

In [ ]:
# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='是否进件')
# df_pivot2 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl', mode='a') as writer:
#         df_pivot2.to_excel(writer, sheet_name='iPhone16系列转化(进件)')

# 待审核

In [ ]:
# df_dsh = df[df['待审核']==1]
# df_dsh = df_dsh[df_dsh['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_dsh = df_dsh[['order_number', 'create_time', 'product_name', '租赁方案', '内存', '颜色', '来源渠道']]
# df_dsh.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'product_name': '型号'}, inplace=True)
# df_dsh


# with pd.ExcelWriter('F:/需求/iPhone16.xlsx', engine='openpyxl') as writer:
#     df_dsh.to_excel(writer, sheet_name='待审核', index=False)